In [1]:
import pandas as pd
import os
# Define the directory containing the CSV files
data_dir = 'data/'

# Load the tackles data
tackles_file = os.path.join(data_dir, 'filtered_tackles_no_assists.csv')
tackles_data = pd.read_csv(tackles_file)

# Load the games data
games_file = os.path.join(data_dir, 'games.csv')
games_data = pd.read_csv(games_file)

# Merge the tackles and games data on gameId and playId
merged_data = tackles_data.merge(games_data[['gameId', 'week']], on=['gameId'], how='left')

# Save the merged data to a new CSV file
merged_data.to_csv(os.path.join(data_dir, 'tackles_with_week.csv'), index=False)


In [14]:
import pandas as pd
import os

# Define the directory containing the CSV files
data_dir = 'data/'

# Load the tackles data
tackles_file = os.path.join(data_dir, 'tackles_with_week.csv')
tackles_data = pd.read_csv(tackles_file)

# Limit the number of rows to process (first 100)
limit = 20

# Function to get player info from tracking data
def get_player_info(game_id, play_id, nfl_id, week):
    # Load the corresponding tracking data for the week
    tracking_file = os.path.join(data_dir, f'filtered_tracking_week_{week}.csv')
    tracking_data = pd.read_csv(tracking_file)

    # Find the rows in tracking_data that match the playId, gameId, and nflId for the player
    player_info = tracking_data[(tracking_data['playId'] == play_id) &
                               (tracking_data['gameId'] == game_id) &
                               (tracking_data['nflId'] == int(nfl_id))]

    if player_info.empty:
        print(f'No tracking data found for player {nfl_id} in play {play_id} of game {game_id} in week {week}')
        player_info = {'x': 0, 'y': 0, 'a': 0, 's': 0, 'dir': 0}
        return None

    # Get the 19th last row or the last available row if there are fewer rows
    player_info = player_info.iloc[-min(19, len(player_info))]
    return player_info
# Add player info to tackles data (even if player_info is None)
for index, row in tackles_data.iterrows():
    if index >= limit:
        break   # Stop processing after reaching the limit

    game_id = row['gameId']
    play_id = row['playId']
    nfl_id = row['nflId']
    week = row['week']

    player_info = get_player_info(game_id, play_id, nfl_id, week)
    print(player_info['x'])
    row['player_x'] = player_info['x']
    row['player_y'] = player_info['y']
    row['player_a'] = player_info['a']
    row['player_s'] = player_info['s']
    row['player_dir'] = player_info['dir']

# Save the updated tackles data to a new CSV file
tackles_data.to_csv(os.path.join(data_dir, 'tackles_with_info.csv'), index=False)


68.42
51.69
63.8
59.76
80.71
39.2
39.42
71.29
84.59
76.31
80.74
77.77
80.24
42.6
61.6
55.65
71.21
94.36
74.81
76.42


In [19]:
import pandas as pd

# Read the main file, limiting to the first 20 rows
tackles_df = pd.read_csv('data/tackles_with_week.csv', nrows=20)

# Get the total number of rows for progress tracking
total_rows = len(tackles_df)

# Prepare a list to hold the extended rows
extended_rows = []

# Iterate through each row in the tackles file
for index, row in tackles_df.iterrows():
    # Print progress
    print(f"Processing row {index + 1} of {total_rows}")

    # Construct the filename for the tracking data
    tracking_filename = f"data/filtered_tracking_week_{row['week']}.csv"
    
    # Read the tracking data for the corresponding week
    tracking_df = pd.read_csv(tracking_filename)
    
    # Filter the tracking data to find the matching rows
    matching_rows = tracking_df[
        (tracking_df['gameId'] == row['gameId']) &
        (tracking_df['playId'] == row['playId']) &
        (tracking_df['nflId'] == row['nflId'])
    ]
    
    # Check if there are enough rows to extract the 19th last one
    if len(matching_rows) >= 19:
        # Get the 19th last row
        selected_row = matching_rows.iloc[-19]
        
        # Extract the required columns
        extended_data = {
            'x': selected_row['x'],
            'y': selected_row['y'],
            's': selected_row['s'],
            'a': selected_row['a'],
            'dir': selected_row['dir']
        }
        
        # Append the extended data to the current row
        extended_row = {**row.to_dict(), **extended_data}
        extended_rows.append(extended_row)

# Create a DataFrame from the extended rows
extended_df = pd.DataFrame(extended_rows)

# Write the extended DataFrame to a new CSV file
extended_df.to_csv('data/extended_tackles_with_tracking_test_batch.csv', index=False)

print("Processing complete.")


Processing row 1 of 20
Processing row 2 of 20
Processing row 3 of 20
Processing row 4 of 20
Processing row 5 of 20
Processing row 6 of 20
Processing row 7 of 20
Processing row 8 of 20
Processing row 9 of 20
Processing row 10 of 20
Processing row 11 of 20
Processing row 12 of 20
Processing row 13 of 20
Processing row 14 of 20
Processing row 15 of 20
Processing row 16 of 20
Processing row 17 of 20
Processing row 18 of 20
Processing row 19 of 20
Processing row 20 of 20
Processing complete.


In [20]:
import pandas as pd

# Read the main file
tackles_df = pd.read_csv('data/tackles_with_week.csv')

# Read the plays data
plays_df = pd.read_csv('data/plays.csv')

# Merge the tackles data with the plays data on gameId and playId
merged_df = pd.merge(tackles_df, plays_df[['gameId', 'playId', 'ballCarrierId']], on=['gameId', 'playId'], how='left')

# Write the merged DataFrame to a new CSV file
merged_df.to_csv('data/tackles_with_ballCarrierId.csv', index=False)

print("Added ballCarrierId and saved to 'tackles_with_ballCarrierId.csv'")


Added ballCarrierId and saved to 'tackles_with_ballCarrierId.csv'


In [1]:
import pandas as pd

def get_tracking_data(tracking_df, gameId, playId, playerId, prefix):
    # Filter the tracking data to find the matching rows
    matching_rows = tracking_df[
        (tracking_df['gameId'] == gameId) &
        (tracking_df['playId'] == playId) &
        (tracking_df['nflId'] == playerId)
    ]
    
    # Check if there are enough rows to extract the 19th last one
    if len(matching_rows) >= 19:
        # Get the 19th last row
        selected_row = matching_rows.iloc[-19]
    else:
        # If not enough rows, use zeroes
        selected_row = pd.Series([0, 0, 0, 0, 0], index=['x', 'y', 's', 'a', 'dir'])

    # Extract the required columns with the given prefix
    return {
        f'{prefix}_x': selected_row['x'],
        f'{prefix}_y': selected_row['y'],
        f'{prefix}_s': selected_row['s'],
        f'{prefix}_a': selected_row['a'],
        f'{prefix}_dir': selected_row['dir']
    }




In [4]:
import pandas as pd
import concurrent.futures

def process_row(row):
    # Read the tracking data for the corresponding week
    tracking_filename = f"data/tracking_week_{row['week']}.csv"
    tracking_df = pd.read_csv(tracking_filename)

    # Get tracking data for nflId
    player_data = get_tracking_data(tracking_df, row['gameId'], row['playId'], row['nflId'], 'player')

    # Get tracking data for ballCarrierId
    ball_data = get_tracking_data(tracking_df, row['gameId'], row['playId'], row['ballCarrierId'], 'ball')

    # Combine data
    extended_row = {**row, **player_data, **ball_data}
    return extended_row

# Main execution function
def main():
    # Read the main files
    tackles_df = pd.read_csv('data/tackles_with_week.csv', nrows=200)
    plays_df = pd.read_csv('data/plays.csv')

    # Merge the tackles data with the plays data on gameId and playId
    merged_df = pd.merge(tackles_df, plays_df[['gameId', 'playId', 'ballCarrierId']], on=['gameId', 'playId'], how='left')

    # Process rows in parallel
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(process_row, merged_df.to_dict('records')))
    
    # Create a DataFrame from the extended rows
    extended_df = pd.DataFrame(results)

    # Write the extended DataFrame to a new CSV file
    extended_df.to_csv('data/extended_tackles_with_tracking_test_batch_200.csv', index=False)

    print("Test processing complete. Data saved to 'extended_tackles_with_tracking_all.csv'")

if __name__ == '__main__':
    main()


Test processing complete. Data saved to 'extended_tackles_with_tracking_all.csv'


In [3]:
import pandas as pd

def remove_zero_ball_dir_rows(csv_file, output_file=None):
    # Read the CSV file
    data = pd.read_csv(csv_file)

    # Filter out rows where ball_dir is 0.0
    filtered_data = data[data['ball_dir'] != 0.0]

    if output_file:
        # Save the filtered data to a new CSV file
        filtered_data.to_csv(output_file, index=False)
        print(f"Filtered data saved to {output_file}")
    else:
        # Return the filtered DataFrame
        return filtered_data

# Example usage
csv_file = 'data/extended_tackles_with_tracking.csv'  # Replace with your CSV file path
output_file = 'data/extended_tackles_with_tracking.csv'  # Replace with your desired output file path

# To save the filtered data to a new CSV file
remove_zero_ball_dir_rows(csv_file, output_file)

# Or to get the filtered data as a DataFrame (without saving to a file)
# filtered_df = remove_zero_ball_dir_rows(csv_file)
# print(filtered_df)


Filtered data saved to data/extended_tackles_with_tracking.csv


In [5]:
import pandas as pd

# Load the dataset
df = pd.read_csv('data/extended_tackles_with_tracking_test_batch_200.csv')

# Group by 'gameId' and 'playId', then count unique 'nflId' and 'ballCarrierId'
grouped = df.groupby(['gameId', 'playId']).agg({'nflId': pd.Series.nunique, 'ballCarrierId': pd.Series.nunique})

# Sum the counts of unique 'nflId' and 'ballCarrierId' for each group
grouped['total_unique_players'] = grouped['nflId'] + grouped['ballCarrierId']

# Find the maximum count of unique players across all groups
max_unique_players = grouped['total_unique_players'].max()

print(f"Maximum number of unique players involved in a single play: {max_unique_players}")

Maximum number of unique players involved in a single play: 2


In [8]:
import pandas as pd

# Load your CSV file into a Pandas DataFrame with explicit dtype
csv_file_path = 'data/extended_tackles_with_tracking_test_batch_200.csv'
data = pd.read_csv(csv_file_path, dtype={'tackle': int})

#Get unique values from the 'tackle' column
unique_tackle_values = data['tackle'].unique()

# Print the unique values
for value in unique_tackle_values:
    print(value)


1


In [1]:
import csv

# Function to merge rows from two CSV files
def merge_csv(input_file, output_file):
    # Initialize lists to store the selected rows
    first_100_rows = []
    rows_after_8777 = []

    with open(input_file, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        for i, row in enumerate(reader):
            if i < 500:
                first_100_rows.append(row)
            elif i >= 8777 and i < 9277:
                rows_after_8777.append(row)
    
    # Merge the selected rows
    merged_rows = first_100_rows + rows_after_8777

    # Write the merged rows to a new CSV file
    with open(output_file, 'w', newline='') as new_csvfile:
        writer = csv.writer(new_csvfile)
        writer.writerows(merged_rows)

# Input and output file paths
input_file_path = 'data/extended_tackles_with_tracking_full.csv'  # Replace with your input CSV file path
output_file_path = 'data/extended_tackles_with_tracking_1000.csv'  # Replace with the desired output CSV file path

# Call the merge_csv function
merge_csv(input_file_path, output_file_path)

print(f"The first 100 rows and 100 rows after row 8777 have been merged into '{output_file_path}'.")


The first 100 rows and 100 rows after row 8777 have been merged into 'data/extended_tackles_with_tracking_mixed.csv'.


In [3]:
import csv

# Function to print the maximum value for each column
def print_max_per_column(csv_file):
    with open(csv_file, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader)  # Read and skip the header row
        max_values = [float('-inf')] * len(header)  # Initialize with negative infinity

        for row in reader:
            for i, value in enumerate(row):
                max_values[i] = max(max_values[i], float(value))

    print("Maximum value for each column:")
    for i, header_value in enumerate(header):
        print(f"{header_value}: {max_values[i]}")

# Specify the path to your CSV file
csv_file_path = 'data/extended_tackles_with_tracking_full.csv'  # Replace with the actual path to your CSV file

# Call the function to print maximum values per column
print_max_per_column(csv_file_path)


Maximum value for each column:
gameId: 2022110700.0
playId: 5096.0
nflId: 55241.0
tackle: 1.0
assist: 0.0
forcedFumble: 1.0
pff_missedTackle: 1.0
week: 9.0
ballCarrierId: 55158.0
player_x: 113.4
player_y: 56.84
player_s: 10.89
player_a: 8.35
player_dir: 359.98
ball_x: 117.25
ball_y: 53.17
ball_s: 10.48
ball_a: 8.63
ball_dir: 359.96


In [5]:
import pandas as pd
import numpy as np

# Load the CSV file into a Pandas DataFrame
csv_file_path = 'data/extended_tackles_with_tracking_full.csv'
df = pd.read_csv(csv_file_path)

# Calculate the Euclidean distance between (player_x, player_y) and (ball_x, ball_y) for each row
df['distance'] = np.sqrt((df['player_x'] - df['ball_x'])**2 + (df['player_y'] - df['ball_y'])**2)

# Find the row with the largest distance
max_distance_row = df.loc[df['distance'].idxmax()]

# Print the row with the largest distance and the distance itself
print("Row with the largest distance:")
print(max_distance_row)
print(f"Largest distance: {max_distance_row['distance']}")


Row with the largest distance:
gameId              2.022103e+09
playId              3.507000e+03
nflId               4.492500e+04
tackle              0.000000e+00
assist              0.000000e+00
forcedFumble        0.000000e+00
pff_missedTackle    1.000000e+00
week                8.000000e+00
ballCarrierId       4.791100e+04
player_x            5.163000e+01
player_y            3.317000e+01
player_s            6.500000e-01
player_a            1.450000e+00
player_dir          4.287000e+01
ball_x              9.778000e+01
ball_y              5.262000e+01
ball_s              9.200000e+00
ball_a              1.610000e+00
ball_dir            9.108000e+01
distance            5.008118e+01
Name: 9634, dtype: float64
Largest distance: 50.08118409143298
